# Part 3

# Setup

In [1]:
using Plots
using JuMP, HiGHS

include("utils.jl")
include("extended_form.jl")
include("scenario_tree.jl")

scenario_tree (generic function with 1 method)

In [2]:
DATA_DIR = "data/"

"data/"

In [ ]:
prices = vec(read_file(DATA_DIR * "belpex_price.txt"))
states = vec(read_file(DATA_DIR * "markov_support_4.txt"))
P_prob = read_file(DATA_DIR * "markov_weights_4.txt")

4×4 Matrix{Float64}:
 0.394106    0.524075   0.0807228  0.00109546
 0.107876    0.564185   0.311323   0.016616
 0.016616    0.311323   0.564185   0.107876
 0.00109546  0.0807228  0.524075   0.394106

# Questions

Now, consider the full multistge stochastic model

### 1. How many nodes are there in the full scenario tree ?

For N possible different states and T time steps, we have only one node at $t=1$ (assumed to be in state 1). At $t=2$, we then have $N$ possible outcomes (i.e children in the tree), whose transition probabilities are depending on the Markov chain, so at $t=2$, we have $N$ nodes. At $t=3$, all the possible nodes at $t=2$ also have $N$ children each, so $N^2$ nodes in total. We continue like this until $t=T$, where there are $N^{T-1}$ children nodes. In total, there are $\sum_{t=0}^{T-1} N^t$ nodes, which is equal to $\frac{N^T-1}{N-1}$ nodes. Setting $T=24$ and $N=4$, we then obtain $\frac{4^{24}-1}{3}$ nodes which is $\approx 9.38 \times 10^{13}$ nodes.

### 2. Implement both the extended formulation of the multistage stochastic program and the scenario tree formulation for $H < T$. Solve both formulations for $H=1,2,\dots$ until you reach the memory limit of your computer. What is the maximum $\bar{H}$ that your computer can handle ?

The maximal H that my computer can handle is 11.

In [4]:
N = 4
H = 4

4

In [5]:
scenarios = generate_scenarios(H, N)
num_scenarios = length(scenarios)

λ_paths = Vector{Vector{Float64}}(undef, num_scenarios)
probs = zeros(num_scenarios)
for (i, s) in enumerate(scenarios)
    λ_paths[i], probs[i] = compute_price_path(H,s,prices,states, P_prob)
end

m_ext = extended_multistage(λ_paths, H, num_scenarios)
optimize!(m_ext)

println("Objective value: ", objective_value(m_ext))

Set parameter Username
Set parameter LicenseID to value 2587316
Academic license - for non-commercial use only - expires 2025-11-19
Objective value: 21721.778592498576


In [6]:
tree = build_tree(H, N, P_prob)
m_tree = scenario_tree(states, prices, tree)

optimize!(m_tree)
println("Objective value: ", objective_value(m_tree))

Set parameter Username
Set parameter LicenseID to value 2587316
Academic license - for non-commercial use only - expires 2025-11-19
Objective value: 14395.98649410305


### 3. Plot the evolution of the number of variables and constraints for both formulations as H increases from $1$ to $\bar{H}$ 

In [7]:
H_vals = 1:11
vars_ext = Int[]
cons_ext = Int[]
vars_tree = Int[]
cons_tree = Int[]

N = 4

for H in H_vals
    # extended
    scenarios = generate_scenarios(H, N)
    println(H)
    num_scenarios = length(scenarios)
    λ_paths = Vector{Vector{Float64}}(undef, num_scenarios)
    probs = zeros(num_scenarios)
    for (i, s) in enumerate(scenarios)
        λ_paths[i], probs[i] = compute_price_path(H, s, prices, states, P_prob)
    end

    model_ext = extended_multistage(λ_paths, H, num_scenarios)
    push!(vars_ext, num_variables(model_ext))
    push!(cons_ext, num_constraints(model_ext; count_variable_in_set_constraints = false))

    # tree
    tree = build_tree(H, N, P_prob)
    model_tree = scenario_tree(states, prices, tree)
    push!(vars_tree, num_variables(model_tree))
    push!(cons_tree, num_constraints(model_tree; count_variable_in_set_constraints = false))
end

cons_ext_clean = [c == 0 ? NaN : c for c in cons_ext]
cons_tree_clean = [c == 0 ? NaN : c for c in cons_tree]

# Plot variables
plt_vars = plot(
    H_vals[1:length(vars_ext)], vars_ext,
    label = "Extended",
    lw = 2, marker = :circle,
    xlabel = "H", ylabel = "Number of Variables",
    title = "Variables vs H",
    yscale =:log10
)
plot!(
    H_vals[1:length(vars_tree)], vars_tree,
    label = "Scenario Tree",
    lw = 2, marker = :square
)
savefig(plt_vars, "variables_vs_H.pdf")


# Plot constraints
plt_cons = plot(
    H_vals[1:length(cons_ext)], cons_ext_clean,
    label = "Extended",
    lw = 2, marker = :circle,
    xlabel = "H", ylabel = "Number of Constraints",
    title = "Constraints vs H",
    yscale =:log10
)
plot!(
    H_vals[1:length(cons_tree_clean)], cons_tree_clean,
    label = "Scenario Tree",
    lw = 2, marker = :square
)
savefig(plt_cons, "constraints_vs_H.pdf")

1
Set parameter Username
Set parameter LicenseID to value 2587316
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Set parameter LicenseID to value 2587316
Academic license - for non-commercial use only - expires 2025-11-19
2
Set parameter Username
Set parameter LicenseID to value 2587316
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Set parameter LicenseID to value 2587316
Academic license - for non-commercial use only - expires 2025-11-19
3
Set parameter Username
Set parameter LicenseID to value 2587316
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Set parameter LicenseID to value 2587316
Academic license - for non-commercial use only - expires 2025-11-19
4
Set parameter Username
Set parameter LicenseID to value 2587316
Academic license - for non-commercial use only - expires 2025-11-19
Set parameter Username
Set parameter LicenseID to value 2587316
Acad

"c:\\Users\\arthu\\OneDrive\\Bureau\\EPL Q10\\LINMA2491-OP_Research\\Projet\\Operational-Research\\constraints_vs_H.pdf"